# Grid Search Practice

Grid search is a way of finding better hypermeters, those that define the configuration of the model and not altered by learning of the model.
It is also known as Exhaustive Grid Search, because it takes many parameter options and creates a search space with cartesian product and then exhaust and evaluate all of these possibilities.
In essence, it is a brute force algorithm.
Let's get familiarize with its usage through practice.

First, we prepare some data.

In [ ]:
from time import time
import os, itertools
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

DATASET = '/dsa/data/all_datasets/titanic_ML/titanic.csv'
assert os.path.exists(DATASET)

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

dataset = pd.read_csv(DATASET).sample(frac = 1).reset_index(drop=True)
X = dataset.iloc[:, :-1]
y = dataset.survived



## Parameter Grid

As a model selection approach, the grid search allows you to investigate the optimal choice of model parameters by specifying the variations for each parameter.

For example, the SVC model has an error penalty parameter `C` in the model,
and we can specify the parameter grid as a dictionary, with the name of the parameter in question, then define the variations of the parameter in a list:

```python
param_grid = {'C': [1e3, 1e4] }
```

The dictionary allows us to supply the variations for different parameters:

```python
param_grid = {'C': [1e3, 1e4],
              'gamma': [1e-4, 1e-3], }
```

Conceptually, the parameter grid dictionary represents a cartesian product of the parameter variations, which contains all the configurations of the models that will be evaluated.  
In other words, the above `param_grid` helps to set up the following 4 models to evaluate:

```python
SVC(C=1e3, gamma=1e-4, ...)
SVC(C=1e3, gamma=1e-3, ...)
SVC(C=1e4, gamma=1e-4, ...)
SVC(C=1e4, gamma=1e-3, ...)
```

But sometimes this kind of cartesian product will generate too many models, more than desired.
We can also provide multiply dictionaries as alternative options.

Consider the difference between
```python
param_grid = [{'C': [1e3, 3e3],
              'gamma': [1e-4, 1e-3], },
             {'C': [5e2, 1e3],
              'gamma': [5e-5, 1e-4], }]
=> 2*2+2*2 = 8 models

```

and

```python
param_grid = [{'C': [1e3, 3e3, 5e2],
              'gamma': [1e-4, 1e-3, 5e-5], }]
=> 3*3 = 9 models
```

## Cross Validation

Not only does the Grid Search help you set up various configurations of models, it also sets up cross validation to provide more objective evaluation metrics of these models. `cv` parameter is used to specify number of cross validation folds used for evaluation.

## Practice

With the parameter grid and cross validation, the grid search results in a lot of computation.
However, we can leverage multiple processors available to accelerate the task, with `n_jobs` parameter, although be aware that in your Jupyter server learning environment, there may be resource limits to ensure fairness, i.e. you may not be able to use as many CPU cores as you can see, so we use a relatively low number in that parameter.

Now let's practice GridSearchCV usage, create a GridSearchCV, **named `clf`**:

1. Create at least 8 models with variations in `C`, `gamma` or your choice.
2. Use 5 fold cross validation.
3. Use 2 parallel jobs.

In [ ]:
# Add your code for the above task here:   (Question #P001)
# ----------------------------------------






Fit the model to loaded data `X` and `y`

In [ ]:
# Add your code for the above task here:   (Question #P002)
# ----------------------------------------






In [ ]:
clf.best_estimator_

The GridSearchCV creates an abstract classifier, once trained with cross validation, the returned object itself can be used as a classifier that represent the optimal classifier within the given hyperparameter space.
For example, from the example above, `clf` will have many familiar methods like `.predict()`, `.score()`.

In addition, `clf.best_estimator_` gives you access to the best model chosen; `clf.best_score_` stores the accuracy score for the best model as well; `clf.cv_results_` provides details on cross validation.


In [ ]:
clf.cv_results_

Now from `clf.cv_results_`, can you find where did the value of `clf.best_score_` come from?

Copy-paste the key/value pair from `clf.cv_results_` that shows the source of `clf.best_score_` below:

Which was the fastest model to train? What were the parameters to the fastest model?

Run a prediction on the first 5 data samples from `X`.

In [ ]:
# Enter your answer below   (Question #P005)
# ----------------------------------------





